# POC Smart prospection: sem o places do google

In [ ]:
!pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 5.6 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np

#NLP
from deep_translator import GoogleTranslator
import nltk 
from nltk import tokenize, ngrams
from nltk.corpus import wordnet as wn
import spacy
import spacy.cli

import os
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE

#Geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy.distance

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Para aparecer todas as linhas e colunas do dataset

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Smart_Locator/datasets/CNPJ_metadados/estabelecimentos_MG.csv')
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico
0,37.391.461/0001-42,MATRIZ,REBOQUE RODRIGUES,NaN,20200612,"Transporte rodoviário de carga, exceto produto...",4930201,NaN,RUA,CIDRA,18,CASA;CASA;CASA;CASA,LIMOEIRO,35162854,MG,IPATINGA,31.0,86030816.0,NaN,NaN,VVIDROFLEX@GMAIL.COM
1,37.391.603/0001-71,MATRIZ,ELIMACLEY,NaN,20200612,Comércio varejista de bebidas,4723700,NaN,RUA,DAVID RABELO,1175,NaN,ALIPIO DE MELO,30820575,MG,BELO HORIZONTE,31.0,85797489.0,NaN,NaN,ELIMACLEY@GMAIL.COM
2,37.391.831/0001-41,MATRIZ,CANTINA DE CONVENIENCIA,NaN,20200612,Cantinas - serviços de alimentação privativos,5620103,"5611204,5611203",RUA,OLEGARIO MACIEL,778,NaN,SAO JOSE,35590000,MG,LAGOA DA PRATA,37.0,99470940.0,NaN,NaN,PAULAO.PAULAO2020@BOL.COM.BR
3,37.392.078/0001-09,MATRIZ,NaN,NaN,20181101,Condomínios prediais,8112500,NaN,RUA,PROFESSOR MERCEDO MOREIRA,373,NaN,ITAPOA,31710340,MG,BELO HORIZONTE,31.0,34050208.0,31.0,34270208.0,GREGOFORT.CONDOMINIOS@YAHOO.COM
4,37.392.102/0001-00,MATRIZ,ME RESTAURANTE E LANCHONETE,NaN,20200612,Restaurantes e similares,5611201,"4723700,4721104,5611203",AVENIDA,SAO JUDAS TADEU,1730,LOJA,SANTA CECILIA,35740000,MG,ESMERALDAS,31.0,25553028.0,NaN,NaN,MARQUESCONTABIL@YAHOO.COM.BR


In [ ]:
df.shape

(2350989, 20)

In [ ]:
df.isnull().sum()

cnpj                                 0
identificador_matriz_filial          0
nome_fantasia                   757574
pais                           2350934
data_inicio_atividade                0
cnae_fiscal_principal                0
cnae_fiscal_secundaria         1022627
tipo_logradouro                   6622
logradouro                           8
numero                              21
complemento                    1425654
bairro                            3550
cep                                  0
uf                                   0
municipio                            0
ddd1                            116311
telefone1                       116325
ddd2                           2157820
telefone2                      2158335
correio_eletronico              369366
dtype: int64

In [ ]:
df['municipio'].value_counts()

BELO HORIZONTE                    434821
UBERLANDIA                        109624
CONTAGEM                           85677
JUIZ DE FORA                       74775
BETIM                              46685
MONTES CLAROS                      46388
UBERABA                            38576
DIVINOPOLIS                        37872
GOVERNADOR VALADARES               33395
IPATINGA                           31189
RIBEIRAO DAS NEVES                 27886
SETE LAGOAS                        27885
POCOS DE CALDAS                    23153
PATOS DE MINAS                     22264
SANTA LUZIA                        20943
POUSO ALEGRE                       19778
VARGINHA                           18863
NOVA LIMA                          16434
NOVA SERRANA                       16328
IBIRITE                            15397
MURIAE                             13985
CONSELHEIRO LAFAIETE               13806
TEOFILO OTONI                      13619
ARAXA                              12841
ARAGUARI        

## Geopy

### Codificando endereços

In [ ]:
df['endereco'] = df['tipo_logradouro'] + ' ' + df['logradouro'] + ', ' + df['municipio'] + ', ' + df['uf'] + ', BRASIL'

In [ ]:
df['endereco'] = df['endereco'].str.lower()

In [ ]:
#df.drop(columns=['endereco', 'endereco_geolocalizado'], inplace=True)

In [ ]:
df.head(20)

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco
0,37.391.461/0001-42,MATRIZ,REBOQUE RODRIGUES,NaN,20200612,"Transporte rodoviário de carga, exceto produto...",4930201,NaN,RUA,CIDRA,18,CASA;CASA;CASA;CASA,LIMOEIRO,35162854,MG,IPATINGA,31.0,86030816.0,NaN,NaN,VVIDROFLEX@GMAIL.COM,"rua cidra, ipatinga, mg, brasil"
1,37.391.603/0001-71,MATRIZ,ELIMACLEY,NaN,20200612,Comércio varejista de bebidas,4723700,NaN,RUA,DAVID RABELO,1175,NaN,ALIPIO DE MELO,30820575,MG,BELO HORIZONTE,31.0,85797489.0,NaN,NaN,ELIMACLEY@GMAIL.COM,"rua david rabelo, belo horizonte, mg, brasil"
2,37.391.831/0001-41,MATRIZ,CANTINA DE CONVENIENCIA,NaN,20200612,Cantinas - serviços de alimentação privativos,5620103,"5611204,5611203",RUA,OLEGARIO MACIEL,778,NaN,SAO JOSE,35590000,MG,LAGOA DA PRATA,37.0,99470940.0,NaN,NaN,PAULAO.PAULAO2020@BOL.COM.BR,"rua olegario maciel, lagoa da prata, mg, brasil"
3,37.392.078/0001-09,MATRIZ,NaN,NaN,20181101,Condomínios prediais,8112500,NaN,RUA,PROFESSOR MERCEDO MOREIRA,373,NaN,ITAPOA,31710340,MG,BELO HORIZONTE,31.0,34050208.0,31.0,34270208.0,GREGOFORT.CONDOMINIOS@YAHOO.COM,"rua professor mercedo moreira, belo horizonte,..."
4,37.392.102/0001-00,MATRIZ,ME RESTAURANTE E LANCHONETE,NaN,20200612,Restaurantes e similares,5611201,"4723700,4721104,5611203",AVENIDA,SAO JUDAS TADEU,1730,LOJA,SANTA CECILIA,35740000,MG,ESMERALDAS,31.0,25553028.0,NaN,NaN,MARQUESCONTABIL@YAHOO.COM.BR,"avenida sao judas tadeu, esmeraldas, mg, brasil"
5,37.392.256/0001-00,MATRIZ,NaN,NaN,20200612,"Transporte rodoviário de carga, exceto produto...",4930201,NaN,RUA,RIO BRANCO,656,NaN,PORTO VELHO,35500430,MG,DIVINOPOLIS,37.0,32293300.0,NaN,NaN,PROCESSOS@AZEVEDOCONTABIL.COM.BR,"rua rio branco, divinopolis, mg, brasil"
6,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,5320202,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br..."
7,37.392.437/0001-28,MATRIZ,JULIANA APARECIDA SILVA CARNEVALLI,NaN,20200612,Comércio varejista de artigos do vestuário e a...,4781400,4789001,10A AVENIDA,AVENIDA LUIZ CORREA CARDOSO,10,LOJA 2,TURQUIA,37517000,MG,MARIA DA FE,35.0,36621804.0,NaN,NaN,JCARNEVALLI12@GMAIL.COM,"10a avenida avenida luiz correa cardoso, maria..."
8,37.392.567/0001-60,MATRIZ,NaN,NaN,20200320,Atividades de organizações políticas,9492800,NaN,RODOVIA,BR 040 KM 689,SN,NaN,ZONA RURAL,36270000,MG,RESSAQUINHA,32.0,99571764.0,NaN,NaN,DENILSONDACRUZ@HOTMAIL.COM,"rodovia br 040 km 689, ressaquinha, mg, brasil"
9,37.392.600/0001-52,MATRIZ,NaN,NaN,20191204,Condomínios prediais,8112500,NaN,RUA,ANTONIA DOS REIS PEREIRA,265,NaN,RECANTO DOS BARREIROS,37552464,MG,POUSO ALEGRE,35.0,30250959.0,NaN,NaN,CONTABILDHARMA@CONTABILDHARMA.COM.BR,"rua antonia dos reis pereira, pouso alegre, mg..."


In [ ]:
jardim = df[df['bairro'] == 'JARDIM DE ALA (JUSTINOPOLIS)']
jardim.shape

(201, 22)

In [ ]:
locator = Nominatim(user_agent="myGeocoder")

In [ ]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [ ]:
jardim['endereco_geolocalizado'] = jardim['endereco'].apply(geocode)

<ipython-input-32-b38bfd6bc846>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jardim['endereco_geolocalizado'] = jardim['endereco'].apply(geocode)


Esse método é muito custoso pois demora um certo tempo para fazer a geolocalização.

In [ ]:
jardim.head()

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado
6,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","(Avenida Ouro Verde, Vila Papine, Justinópolis..."
13711,40.035.426/0001-03,MATRIZ,GARAGE'S FOOD,NaN,20201207,Restaurantes e similares,5611203,RUA,PIRATININGA,11,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943630,MG,RIBEIRAO DAS NEVES,31.0,99412433.0,NaN,NaN,GARAGESFOOD@GMAIL.COM,"rua piratininga, ribeirao das neves, mg, brasil","(Rua Piratininga, Jardim de Alá, Justinópolis,..."
22066,41.106.369/0001-60,MATRIZ,PIZZARIA DA LINDAURA,NaN,20210305,Serviços de alimentação para eventos e recepçõ...,NaN,RUA,DO CAMPO,11,CONJ 2,JARDIM DE ALA (JUSTINOPOLIS),33943233,MG,RIBEIRAO DAS NEVES,31.0,85800232.0,NaN,NaN,LINDADARK9@GMAIL.COM,"rua do campo, ribeirao das neves, mg, brasil","(Rua do Campo, Eliane, Justinópolis, Ribeirão ..."
36951,42.665.147/0001-40,MATRIZ,EBENEZER A&V PINTURAS,NaN,20210711,Serviços de pintura de edifícios em geral,4741500,RUA,CURVELO,315,CASA 2,JARDIM DE ALA (JUSTINOPOLIS),33943770,MG,RIBEIRAO DAS NEVES,31.0,87538949.0,NaN,NaN,AMAURIAURELINOAURELINO@GMAIL.COM,"rua curvelo, ribeirao das neves, mg, brasil","(Rua Curvelo, Vila Papine, Justinópolis, Ribei..."
50785,44.080.036/0001-06,MATRIZ,ACAI DRINKS,NaN,20211030,Comércio varejista de produtos alimentícios em...,NaN,AVENIDA,OURO VERDE,238,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,75099824.0,NaN,NaN,FA8361272@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","(Avenida Ouro Verde, Vila Papine, Justinópolis..."


In [ ]:
jardim['endereco_geolocalizado'].isna().sum()

6

In [ ]:
jardim.shape

(201, 22)

In [ ]:
jardim['latitude'] = jardim['endereco_geolocalizado'].apply(lambda loc: loc.latitude if loc else None)
jardim['longitude'] = jardim['endereco_geolocalizado'].apply(lambda loc: loc.longitude if loc else None)
jardim['geocoder'] = jardim['endereco_geolocalizado'].apply(lambda loc: tuple(loc.point) if loc else None)
jardim.head()

<ipython-input-36-b74fbb05c3aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jardim['latitude'] = jardim['endereco_geolocalizado'].apply(lambda loc: loc.latitude if loc else None)
<ipython-input-36-b74fbb05c3aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jardim['longitude'] = jardim['endereco_geolocalizado'].apply(lambda loc: loc.longitude if loc else None)
<ipython-input-36-b74fbb05c3aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder
6,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","(Avenida Ouro Verde, Vila Papine, Justinópolis...",-19.799550,-44.017191,"(-19.79955, -44.0171908, 0.0)"
13711,40.035.426/0001-03,MATRIZ,GARAGE'S FOOD,NaN,20201207,Restaurantes e similares,5611203,RUA,PIRATININGA,11,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943630,MG,RIBEIRAO DAS NEVES,31.0,99412433.0,NaN,NaN,GARAGESFOOD@GMAIL.COM,"rua piratininga, ribeirao das neves, mg, brasil","(Rua Piratininga, Jardim de Alá, Justinópolis,...",-19.806701,-44.021607,"(-19.8067008, -44.0216066, 0.0)"
22066,41.106.369/0001-60,MATRIZ,PIZZARIA DA LINDAURA,NaN,20210305,Serviços de alimentação para eventos e recepçõ...,NaN,RUA,DO CAMPO,11,CONJ 2,JARDIM DE ALA (JUSTINOPOLIS),33943233,MG,RIBEIRAO DAS NEVES,31.0,85800232.0,NaN,NaN,LINDADARK9@GMAIL.COM,"rua do campo, ribeirao das neves, mg, brasil","(Rua do Campo, Eliane, Justinópolis, Ribeirão ...",-19.814770,-44.020641,"(-19.8147701, -44.0206408, 0.0)"
36951,42.665.147/0001-40,MATRIZ,EBENEZER A&V PINTURAS,NaN,20210711,Serviços de pintura de edifícios em geral,4741500,RUA,CURVELO,315,CASA 2,JARDIM DE ALA (JUSTINOPOLIS),33943770,MG,RIBEIRAO DAS NEVES,31.0,87538949.0,NaN,NaN,AMAURIAURELINOAURELINO@GMAIL.COM,"rua curvelo, ribeirao das neves, mg, brasil","(Rua Curvelo, Vila Papine, Justinópolis, Ribei...",-19.803343,-44.020771,"(-19.8033429, -44.0207707, 0.0)"
50785,44.080.036/0001-06,MATRIZ,ACAI DRINKS,NaN,20211030,Comércio varejista de produtos alimentícios em...,NaN,AVENIDA,OURO VERDE,238,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,75099824.0,NaN,NaN,FA8361272@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","(Avenida Ouro Verde, Vila Papine, Justinópolis...",-19.799550,-44.017191,"(-19.79955, -44.0171908, 0.0)"


In [ ]:
jardim.to_csv('bairro_jardim_geolocalizado.csv', index=False)

### Distância entre dois pontos geográficos

In [ ]:
p1 = (-23.5842844, -46.6610056) 
p2 = (-23.6388952, -46.6209493) 
 
print(geopy.distance.distance(p1, p2).km)

7.300209660192287


## Distância entre CEPs

In [ ]:
jardim = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Smart_Locator/datasets/CNPJ_metadados/bairro_jardim_geolocalizado.csv')
jardim.head()

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder
0,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.799550,-44.017191,"(-19.79955, -44.0171908, 0.0)"
1,40.035.426/0001-03,MATRIZ,GARAGE'S FOOD,NaN,20201207,Restaurantes e similares,5611203,RUA,PIRATININGA,11,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943630,MG,RIBEIRAO DAS NEVES,31.0,99412433.0,NaN,NaN,GARAGESFOOD@GMAIL.COM,"rua piratininga, ribeirao das neves, mg, brasil","Rua Piratininga, Jardim de Alá, Justinópolis, ...",-19.806701,-44.021607,"(-19.8067008, -44.0216066, 0.0)"
2,41.106.369/0001-60,MATRIZ,PIZZARIA DA LINDAURA,NaN,20210305,Serviços de alimentação para eventos e recepçõ...,NaN,RUA,DO CAMPO,11,CONJ 2,JARDIM DE ALA (JUSTINOPOLIS),33943233,MG,RIBEIRAO DAS NEVES,31.0,85800232.0,NaN,NaN,LINDADARK9@GMAIL.COM,"rua do campo, ribeirao das neves, mg, brasil","Rua do Campo, Eliane, Justinópolis, Ribeirão d...",-19.814770,-44.020641,"(-19.8147701, -44.0206408, 0.0)"
3,42.665.147/0001-40,MATRIZ,EBENEZER A&V PINTURAS,NaN,20210711,Serviços de pintura de edifícios em geral,4741500,RUA,CURVELO,315,CASA 2,JARDIM DE ALA (JUSTINOPOLIS),33943770,MG,RIBEIRAO DAS NEVES,31.0,87538949.0,NaN,NaN,AMAURIAURELINOAURELINO@GMAIL.COM,"rua curvelo, ribeirao das neves, mg, brasil","Rua Curvelo, Vila Papine, Justinópolis, Ribeir...",-19.803343,-44.020771,"(-19.8033429, -44.0207707, 0.0)"
4,44.080.036/0001-06,MATRIZ,ACAI DRINKS,NaN,20211030,Comércio varejista de produtos alimentícios em...,NaN,AVENIDA,OURO VERDE,238,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,75099824.0,NaN,NaN,FA8361272@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.799550,-44.017191,"(-19.79955, -44.0171908, 0.0)"


In [ ]:
jardim.shape

(201, 25)

In [ ]:
jardim.cnae_fiscal_principal

0                             Serviços de entrega rápida
1                               Restaurantes e similares
2      Serviços de alimentação para eventos e recepçõ...
3              Serviços de pintura de edifícios em geral
4      Comércio varejista de produtos alimentícios em...
                             ...                        
196    Fabricação de produtos de padaria e confeitari...
197                                   Obras de alvenaria
198                               Fabricação de colchões
199    Atividades de fornecimento de infra-estrutura ...
200    Fabricação de móveis com predominância de madeira
Name: cnae_fiscal_principal, Length: 201, dtype: object

In [ ]:
cep1 = jardim.cep.iloc[0]
cep1

33943700

In [ ]:
lista_diff = []
for cep in jardim.cep:
  diferenca = abs(cep - cep1)
  lista_diff.append(diferenca)

jardim['diferenca'] = lista_diff

diff0 = jardim[jardim['diferenca'] == 0]
diff0.shape

(45, 26)

In [ ]:
diff0

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder,diferenca
0,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
4,44.080.036/0001-06,MATRIZ,ACAI DRINKS,NaN,20211030,Comércio varejista de produtos alimentícios em...,NaN,AVENIDA,OURO VERDE,238,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,75099824.0,NaN,NaN,FA8361272@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
22,41.480.706/0001-85,MATRIZ,NaN,NaN,20210407,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,410,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,92955133.0,NaN,NaN,LUIZANTONIOSOUZA5514@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
33,17.575.538/0001-27,MATRIZ,VANIA CONFECCOES,NaN,20130214,"Confecção, sob medida, de peças do vestuário, ...","4755503,4789001,4781400",AVENIDA,OURO VERDE,274,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,36386066.0,NaN,NaN,capasite@outlook.com,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
42,41.036.806/0001-17,MATRIZ,CRIS MACHADO DOS SANTOS,NaN,20210301,"Confecção, sob medida, de peças do vestuário, ...",9700500,AVENIDA,OURO VERDE,361,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,71534088.0,NaN,NaN,ANABIBIMACHADO25@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
43,41.488.501/0001-46,MATRIZ,NaN,NaN,20210407,Outras atividades auxiliares dos transportes t...,NaN,AVENIDA,OURO VERDE,368,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88427254.0,NaN,NaN,ROBERTOFIDELITY@HOTMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
46,44.280.411/0001-53,MATRIZ,NaN,NaN,20211118,Obras de alvenaria,NaN,AVENIDA,OURO VERDE,159,CASA 2,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,98646056.0,NaN,NaN,CONTABILVEREDAS@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
56,30.743.177/0001-49,MATRIZ,GONCALVES S/A,NaN,20180620,Serviços de entrega rápida,4755502,AVENIDA,OURO VERDE,400,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,83035815.0,NaN,NaN,Silvaleandro5289@gmail.com,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
65,40.635.074/0001-19,MATRIZ,GETBURGUER,NaN,20210129,"Lanchonetes, casas de chá, de sucos e similares",NaN,AVENIDA,OURO VERDE,477,CASA,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,36380437.0,NaN,NaN,JORGENASCIMENTOLINDO@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0
66,42.059.653/0001-96,MATRIZ,TAODIVVA,NaN,20210524,Comércio varejista de artigos do vestuário e a...,4782201,AVENIDA,OURO VERDE,545,CXPST 2,JARD

In [ ]:
diff0.isnull().sum()

cnpj                            0
identificador_matriz_filial     0
nome_fantasia                  11
pais                           45
data_inicio_atividade           0
cnae_fiscal_principal           0
cnae_fiscal_secundaria         24
tipo_logradouro                 0
logradouro                      0
numero                          0
complemento                    25
bairro                          0
cep                             0
uf                              0
municipio                       0
ddd1                            0
telefone1                       0
ddd2                           45
telefone2                      45
correio_eletronico              5
endereco                        0
endereco_geolocalizado          0
latitude                        0
longitude                       0
geocoder                        0
diferenca                       0
dtype: int64

In [ ]:
diff0.latitude.mean()

-19.799549999999993

In [ ]:
longs = [diff0.longitude]
lats = [diff0.latitude]

In [ ]:
longs

 Percebe-se que os endereços com o mesmo cep tem o mesmo geocode - 
 Isso se deve pois o número e complemento nao são colocados

In [ ]:
import folium

meanLat = diff0.latitude.mean()
meanLong = diff0.longitude.mean()

mapObj = folium.Map(location = [meanLat,meanLong], zoom_start = 15)
for i in range(0,diff0.shape[0]): 
    markerObj = folium.Marker(location = [lats[i],longs[i]])
    markerObj.add_to(mapObj)

mapObj

## Similaridade entre CNAE

**Existem duas formas de fazer a similaridade:**


1.   Comparando palavras exatas (similaridade entre a estrutura da palavra) -  esse caso seria entre CNAEs
2.   Comparando palavras sinônimas - esse caso seria entre CNAE e RAMOATIVIDADE





### Preprocessamento dos dados NLP

#### Score de similaridade com Spacy

1. Converta o texto em letras minúsculas (correspondência sem distinção entre maiúsculas e minúsculas).
2. Declare uma variável de lista para armazenar os resultados.
3. Faça um loop sobre cada uma das palavras.
4. Verifique se a palavra encontrada na lista de palavras de parada.
5. Acrescente a palavra à variável de lista.
6. Junte as palavras em uma frase e retorne o resultado.

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nlp = spacy.cli.download("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
def remove_stopwords(text):
    doc = nlp(text.lower()) #1
    result = [] #2
    for token in doc: #3
        if token.text in nlp.Defaults.stop_words: #4
            continue
        result.append(token.text)#5
    return " ".join(result)

In [ ]:
remove_stopwords('A casa é bonita')

'casa é bonita'

###  1. Entre CNAES

Pegar todos os CNAES dos clientes do vendedor e fazer uma comparação entre os CNAES dos estabelecimentos próximos

In [ ]:
nlp = spacy.load("pt_core_news_sm")
doc1 = nlp("Serviços de entrega rápida")
doc2 = nlp("Serviços de entrega rápida")
doc1.similarity(doc2)

1.0

In [ ]:
# Palavras identicas
nltk.edit_distance("Serviços rápida entrega", "Serviços")

15

In [ ]:
jardim.head(1)

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,...,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder,diferenca
0,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,...,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0


In [ ]:
jardim.cnae_fiscal_principal.value_counts()

Obras de alvenaria                                                                         19
Cabeleireiros, manicure e pedicure                                                         10
Reparação e manutenção de equipamentos de comunicação                                       9
Serviços de entrega rápida                                                                  9
Lanchonetes, casas de chá, de sucos e similares                                             5
                                                                                           ..
Fabricação de artigos do vestuário, produzidos em malharias e tricotagens, exceto meias     1
Desenvolvimento de programas de computador sob encomenda                                    1
Agências de viagens                                                                         1
Manutenção e reparação de máquinas-ferramenta                                               1
Fabricação de colchões                                      

In [ ]:
cnaes_of_seach = diff0.cnae_fiscal_principal.value_counts().index.tolist()
cnaes_of_seach

['Serviços de entrega rápida',
 'Obras de alvenaria',
 'Fabricação de móveis com predominância de madeira',
 'Confecção, sob medida, de peças do vestuário, exceto roupas íntimas',
 'Outras atividades auxiliares dos transportes terrestres não especificadas anteriormente',
 'Atividades de estética e outros serviços de cuidados com a beleza',
 'Serviços de manutenção e reparação mecânica de veículos automotores',
 'Fabricação de produtos diversos não especificados anteriormente',
 'Serviços domésticos',
 'Treinamento em desenvolvimento profissional e gerencial',
 'Atividades de organizações religiosas ou filosóficas',
 'Comércio varejista de hortifrutigranjeiros',
 'Aplicação de revestimentos e de resinas em interiores e exteriores',
 'Serviços de lanternagem ou funilaria e pintura de veículos automotores',
 'Instalação e manutenção elétrica',
 'Comércio varejista de animais vivos e de artigos e alimentos para animais de estimação',
 'Comércio a varejo de peças e acessórios novos para veí

In [ ]:
cnae1 = jardim.cnae_fiscal_principal[0]

In [ ]:
cnae1

'Serviços de entrega rápida'

#### CNAE Diferente

In [ ]:
nlp = spacy.load("pt_core_news_sm")
doc1 = nlp("Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns")
doc2 = nlp("Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns")
doc1.similarity(doc2)

1.0

In [ ]:
nlp = spacy.load('pt_core_news_sm')
cnae1 = 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns'
list_cnaes = []
list_score = []
df_end = pd.DataFrame()

for cnae_prospec in cnaes_of_seach:
  doc1 = nlp(cnae_prospec)
  doc2 = nlp(str(cnae1))
  similar = doc1.similarity(doc2)

  if similar >= 0.70:
    list_cnaes.append(cnae_prospec)
    list_score.append(round(similar*100,2)) 
  else:
    cnae = None
  tuples = list(zip(list_cnaes, list_score))
  df_end = pd.DataFrame(tuples, columns=['CNAE', 'Similarity']).sort_values(['Similarity'], ascending=False)
  df_end.drop_duplicates(subset=['Similarity'], inplace=True)
  df_end = df_end.reset_index(drop=True)

df_end

<ipython-input-25-601fa9839065>:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similar = doc1.similarity(doc2)


,CNAE,Similarity
0,"Comércio varejista de mercadorias em geral, co...",100.00
1,Comércio varejista de animais vivos e de artig...,81.94
2,"Fabricação de artigos do vestuário, produzidos...",79.80
3,"Transporte rodoviário de carga, exceto produto...",78.42
4,"Lanchonetes, casas de chá, de sucos e similares",76.60
5,Aplicação de revestimentos e de resinas em int...,75.94
6,Comércio a varejo de peças e acessórios novos ...,75.49
7,Comércio varejista de produtos alimentícios em...,74.91
8,Atividades de estética e outros serviços de cu...,73.04
9,Serviços de manutenção e reparação mecânica de...,71.47


In [ ]:
df_end.CNAE.value_counts()

Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns      1
Comércio varejista de animais vivos e de artigos e alimentos para animais de estimação                                            1
Fabricação de artigos do vestuário, produzidos em malharias e tricotagens, exceto meias                                           1
Transporte rodoviário de carga, exceto produtos perigosos e mudanças, municipal.                                                  1
Lanchonetes, casas de chá, de sucos e similares                                                                                   1
Aplicação de revestimentos e de resinas em interiores e exteriores                                                                1
Comércio a varejo de peças e acessórios novos para veículos automotores                                                           1
Comércio varejista de produtos alimentícios em geral ou especializado em pro

In [ ]:
diff0.head(1)

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder,diferenca
0,37.392.382/0001-56,MATRIZ,NaN,NaN,20200612,Serviços de entrega rápida,NaN,AVENIDA,OURO VERDE,300,APT 202,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,88618906.0,NaN,NaN,NaN,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0


In [ ]:
diff0[diff0['cnae_fiscal_principal'] == 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns']

,cnpj,identificador_matriz_filial,nome_fantasia,pais,data_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd1,telefone1,ddd2,telefone2,correio_eletronico,endereco,endereco_geolocalizado,latitude,longitude,geocoder,diferenca
97,26.866.381/0001-80,MATRIZ,NaN,NaN,20170113,"Comércio varejista de mercadorias em geral, co...",NaN,AVENIDA,OURO VERDE,329,NaN,JARDIM DE ALA (JUSTINOPOLIS),33943700,MG,RIBEIRAO DAS NEVES,31.0,85679255.0,NaN,NaN,CONTABILIDADECASCALHO@GMAIL.COM,"avenida ouro verde, ribeirao das neves, mg, br...","Avenida Ouro Verde, Vila Papine, Justinópolis,...",-19.79955,-44.017191,"(-19.79955, -44.0171908, 0.0)",0


In [ ]:
diff0.shape

(45, 26)

In [ ]:
df_end.shape

(33, 26)

### 2. Entre CNAE e RAMOATIVIDADE

In [ ]:
# Dados do cliente do cliente atrib 'ramoatividade'

ramoatividade = ['FARMACIA/DROGARIA',
'FARMACIA/DROG FEBRAFAR',
'OUTROS AMAPET',
'ALIMENTAR AMAPET',
'CLINICA VETERINARIA',
'PET SHOP',
'CASA AGROPECUARIA',
'CASA DE RACAO',
'COMPRA-AGORA',
'SALAO',
'FUNCIONARIO',
'CIRURGICA',
'PERFUMARIA',
'OUTROS',
'VENDEDOR',
'HOSPITAL/CLINICA',
'ATACADISTA',
'HIPERMECADOS',
'SUPERMERCADO/MERCEARIA',
'DISTRIBUIDORA PERFUMARIA',
'DISTRIBUIDORA MEDICAMENTO',
'ORTOPEDIA',
'FUNCIONARIOS AMAPET']

In [ ]:
nlp = spacy.load('pt_core_news_sm')
list_cnaes = []
list_score = []
list_ramoatv = []
df_end = pd.DataFrame()

for cnae_prospec in cnaes_of_seach:
  doc1 = nlp(cnae_prospec)
  for ramoatv in ramoatividade:
    ramoatv = ramoatv.lower()
    doc2 = nlp(str(ramoatv))
    similar = doc1.similarity(doc2)

    if similar >= 0.70:
      list_cnaes.append(cnae_prospec)
      list_ramoatv.append(ramoatv)
      list_score.append(round(similar*100,2)) 
    else:
      cnae = None

  tuples = list(zip(list_cnaes,list_ramoatv, list_score))
  df_end = pd.DataFrame(tuples, columns=['CNAE', 'RAMOATIVDADE', 'Similarity']).sort_values(['Similarity'], ascending=False)
  df_end.drop_duplicates(subset=['Similarity'], inplace=True)

df_end

<ipython-input-82-6a5aaec70f18>:12: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similar = doc1.similarity(doc2)


,CNAE,RAMOATIVDADE,Similarity


### Comparação entre textos

In [ ]:
!easy_install matplotlib-venn

/bin/bash: easy_install: command not found


In [ ]:
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
t1 = "Comércio varejista de artigos de joalheria	"
t2 = "Comércio varejista de artigos de joalheria	"

vect = CountVectorizer(analyzer = 'word', ngram_range= (1, 2)) 

vocab = vect.fit([t1, t2])

In [ ]:
vocab.vocabulary_

{'comércio': 2,
 'varejista': 8,
 'de': 4,
 'artigos': 0,
 'joalheria': 7,
 'comércio varejista': 3,
 'varejista de': 9,
 'de artigos': 5,
 'artigos de': 1,
 'de joalheria': 6}

In [ ]:
test = vect.fit_transform([t1, t2])

In [ ]:
test = test.toarray()

In [ ]:
print(test)
vocab.vocabulary_ 

[[1 1 1 1 2 1 1 1 1 1]
 [1 1 1 1 2 1 1 1 1 1]]


{'comércio': 2,
 'varejista': 8,
 'de': 4,
 'artigos': 0,
 'joalheria': 7,
 'comércio varejista': 3,
 'varejista de': 9,
 'de artigos': 5,
 'artigos de': 1,
 'de joalheria': 6}

In [ ]:
intersection_list = np.amin(test, axis = 0) # Intersecção
intersection_list

array([1, 1, 1, 1, 2, 1, 1, 1, 1, 1])

In [ ]:
sum = np.sum(intersection_list)  
sum

11

In [ ]:
count = np.sum(test[0]) # Texto base para comparação
count

11

In [ ]:
sum/count

1.0

#### Função que mede similaridade

In [ ]:
def medidor_de_similaridade(text1, text2):
  to_vect = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))
  result = []
  for comentario1 in text1:
    for comentario2 in text2:
      x1, x2 = to_vect.fit_transform([comentario1,comentario2])
      t1, t2 = x1.toarray(), x2.toarray()

      min = np.amin([t1, t2], axis = 0)
      sum = np.sum(min)
      count = np.sum([t1, t2][0])
      to_mean = sum/count
      result.append(to_mean)
  return result

In [ ]:
x1 = ["Comércio varejista de artigos de joalheria"]
x2 = ["joalheria"]
medidor_de_similaridade(x1,x2)

[0.09090909090909091]

In [ ]:
cnae1 = 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns'
list_cnaes = []
list_score = []
df_end = pd.DataFrame()

for cnae_prospec in cnaes_of_seach:
  similar = medidor_de_similaridade([cnae_prospec],[cnae1])
  similar = float(str(similar).replace('[','').replace(']',''))

  if similar >= 0.30:
    list_cnaes.append(cnae_prospec)
    list_score.append(round(similar*100,2)) 
  else:
    cnae = None
  tuples = list(zip(list_cnaes, list_score))
  df_end = pd.DataFrame(tuples, columns=['CNAE', 'Similarity']).sort_values(['Similarity'], ascending=False)
  df_end.drop_duplicates(subset=['Similarity'], inplace=True)
  df_end = df_end.reset_index(drop=True)

df_end

,CNAE,Similarity
0,"Comércio varejista de mercadorias em geral, co...",100.00
1,Comércio varejista de hortifrutigranjeiros,71.43
2,Comércio varejista de artigos de joalheria,54.55
3,Fabricação de móveis com predominância de madeira,46.15
4,Padaria e confeitaria com predominância de rev...,45.45
5,Comércio varejista de produtos alimentícios em...,41.38
6,Serviços de pintura de edifícios em geral,38.46
7,Fabricação de produtos de padaria e confeitari...,38.10


##### Limpando os textos (não tem mudança significativa)

In [ ]:
def clean_function(df, column_text):
  df[column_text] = df[[f'{column_text}']].replace(
      regex=r'[!/,.-]',value='').apply(lambda x: x.astype(str).str.lower()).apply(
      lambda x:x.astype(str).str.normalize('NFKD').str.encode('ascii', 
                                                        errors='ignore').str.decode('utf-8'))
  return df

In [ ]:
clean_cnae = clean_function(diff0,'cnae_fiscal_principal')

<ipython-input-151-06eab07774a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_text] = df[[f'{column_text}']].replace(


In [ ]:
cnaes_clean = clean_cnae.cnae_fiscal_principal.value_counts().index.tolist()
cnaes_clean

['servicos de entrega rapida',
 'obras de alvenaria',
 'fabricacao de moveis com predominancia de madeira',
 'confeccao sob medida de pecas do vestuario exceto roupas intimas',
 'outras atividades auxiliares dos transportes terrestres nao especificadas anteriormente',
 'atividades de estetica e outros servicos de cuidados com a beleza',
 'servicos de manutencao e reparacao mecanica de veiculos automotores',
 'fabricacao de produtos diversos nao especificados anteriormente',
 'servicos domesticos',
 'treinamento em desenvolvimento profissional e gerencial',
 'atividades de organizacoes religiosas ou filosoficas',
 'comercio varejista de hortifrutigranjeiros',
 'aplicacao de revestimentos e de resinas em interiores e exteriores',
 'servicos de lanternagem ou funilaria e pintura de veiculos automotores',
 'instalacao e manutencao eletrica',
 'comercio varejista de animais vivos e de artigos e alimentos para animais de estimacao',
 'comercio a varejo de pecas e acessorios novos para veicul

In [ ]:
cnae1 = 'comercio varejista de mercadorias em geral com predominancia de produtos alimenticios  minimercados mercearias e armazens'
list_cnaes = []
list_score = []

for cnae_prospec in cnaes_clean:
  similar = medidor_de_similaridade([cnae_prospec],[cnae1])
  similar = float(str(similar).replace('[','').replace(']',''))

  if similar >= 0.30:
    list_cnaes.append(cnae_prospec)
    list_score.append(round(similar*100,2)) 
  else:
    cnae = None
  tuples = list(zip(list_cnaes, list_score))
  df_end = pd.DataFrame(tuples, columns=['CNAE', 'Similarity']).sort_values(['Similarity'], ascending=False)
  df_end.drop_duplicates(subset=['Similarity'], inplace=True)
  df_end = df_end.reset_index(drop=True)

df_end

,CNAE,Similarity
0,comercio varejista de mercadorias em geral com...,100.00
1,comercio varejista de hortifrutigranjeiros,71.43
2,comercio varejista de artigos de joalheria,54.55
3,fabricacao de moveis com predominancia de madeira,46.15
4,padaria e confeitaria com predominancia de rev...,45.45
5,comercio varejista de produtos alimenticios em...,41.38
6,servicos de pintura de edificios em geral,38.46
7,fabricacao de produtos de padaria e confeitari...,38.10


## NLP estudos

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 7.4 MB/s eta 0:00:00


In [ ]:
%matplotlib inline
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import nltk 
from nltk import tokenize, ngrams
from wordcloud import WordCloud
from string import punctuation
import unidecode

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
phrases = ["comercio varejista de mercadorias em geral com predominancia de produtos alimenticios  minimercados mercearias e armazens", 
           "comercio varejista de produtos alimenticios em geral ou especializado em produtos alimenticios nao especificados anteriormente"]

tfidf = TfidfVectorizer(lowercase=False, max_features=50)

characteristics = tfidf.fit_transform(phrases)
# Other way to do a sparse matrix
pd.DataFrame(
    characteristics.todense(),
    columns=tfidf.get_feature_names()
)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,alimenticios,anteriormente,armazens,com,comercio,de,em,especializado,especificados,geral,mercadorias,mercearias,minimercados,nao,ou,predominancia,produtos,varejista
0,0.213922,0.00000,0.300659,0.300659,0.213922,0.427843,0.213922,0.00000,0.00000,0.213922,0.300659,0.300659,0.300659,0.00000,0.00000,0.300659,0.213922,0.213922
1,0.393166,0.27629,0.000000,0.000000,0.196583,0.196583,0.393166,0.27629,0.27629,0.196583,0.000000,0.000000,0.000000,0.27629,0.27629,0.000000,0.393166,0.196583


In [ ]:
stopwords_without_accents = [unidecode.unidecode(text) for text in phrases]
stopwords_without_accents

['comercio varejista de mercadorias em geral com predominancia de produtos alimenticios  minimercados mercearias e armazens',
 'comercio varejista de produtos alimenticios em geral ou especializado em produtos alimenticios nao especificados anteriormente']

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')
stopword = nltk.corpus.stopwords.words("portuguese")
stemmer = nltk.stem.RSLPStemmer()
stemmer.stem("correria")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


'corr'

In [ ]:
stemmer.stem(phrases[0])

In [ ]:
token_space = tokenize.WhitespaceTokenizer()

phrase = phrases[0]
separate_phrases = token_space.tokenize(phrase)
evens = ngrams(separate_phrases, 2) # bi-gram in this case
list(evens)

[('comercio', 'varejista'),
 ('varejista', 'de'),
 ('de', 'mercadorias'),
 ('mercadorias', 'em'),
 ('em', 'geral'),
 ('geral', 'com'),
 ('com', 'predominancia'),
 ('predominancia', 'de'),
 ('de', 'produtos'),
 ('produtos', 'alimenticios'),
 ('alimenticios', 'minimercados'),
 ('minimercados', 'mercearias'),
 ('mercearias', 'e'),
 ('e', 'armazens')]

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
texto = 'comercio varejista de mercadorias em geral com predominancia de produtos alimenticios  minimercados mercearias e armazens'

In [ ]:
sentences = nltk.sent_tokenize(texto)

In [ ]:
for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i])
    newwords = [word for word in words if word not in stopwords.words('portuguese')]
    sentences[i] = ' '.join(newwords)
    
print (sentences)

['comercio varejista mercadorias geral predominancia produtos alimenticios minimercados mercearias armazens']


In [ ]:
sentences[0]

'comercio varejista mercadorias geral predominancia produtos alimenticios minimercados mercearias armazens'

In [ ]:
token_space = tokenize.WhitespaceTokenizer()
separate_phrases = token_space.tokenize(sentences[0])
evens = ngrams(separate_phrases, 2) # bi-gram in this case
bigrams = list(evens)

In [ ]:
bigrams

[('comercio', 'varejista'),
 ('varejista', 'mercadorias'),
 ('mercadorias', 'geral'),
 ('geral', 'predominancia'),
 ('predominancia', 'produtos'),
 ('produtos', 'alimenticios'),
 ('alimenticios', 'minimercados'),
 ('minimercados', 'mercearias'),
 ('mercearias', 'armazens')]

In [ ]:
bigrams[0][0]

'comercio'

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences[0], min_count=1)

words = model.wv.vocab

vector = model.wv['supermercado']

# busca a palavra mais similar
similar = model.wv.most_similar('supermercado')
print (similar)

KeyError: ignored

In [ ]:
words

{'c': <gensim.models.keyedvectors.Vocab at 0x7ff00ffb2370>,
 'o': <gensim.models.keyedvectors.Vocab at 0x7ff00fd38940>,
 'm': <gensim.models.keyedvectors.Vocab at 0x7ff00fd387f0>,
 'e': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61070>,
 'r': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61100>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61160>,
 ' ': <gensim.models.keyedvectors.Vocab at 0x7ff00fa611c0>,
 'v': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61220>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61280>,
 'j': <gensim.models.keyedvectors.Vocab at 0x7ff00fa612e0>,
 's': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61340>,
 't': <gensim.models.keyedvectors.Vocab at 0x7ff00fa613a0>,
 'd': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61400>,
 'g': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61460>,
 'l': <gensim.models.keyedvectors.Vocab at 0x7ff00fa614c0>,
 'p': <gensim.models.keyedvectors.Vocab at 0x7ff00fa61520>,
 'n': <gensim.models.keyedvectors.Vocab 